In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('nearest-earth-objects(1910-2024).csv')

In [3]:
df.head()

,neo_id,name,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,orbiting_body,relative_velocity,miss_distance,is_hazardous
0,2162117,162117 (1998 SD15),19.14,0.394962,0.883161,Earth,71745.401048,5.814362e+07,False
1,2349507,349507 (2008 QY),18.50,0.530341,1.185878,Earth,109949.757148,5.580105e+07,True
2,2455415,455415 (2003 GA),21.45,0.136319,0.304818,Earth,24865.506798,6.720689e+07,False
3,3132126,(2002 PB),20.63,0.198863,0.444672,Earth,78890.076805,3.039644e+07,False
4,3557844,(2011 DW),22.70,0.076658,0.171412,Earth,56036.519484,6.311863e+07,False


In [4]:
df.orbiting_body.unique()

array(['Earth'], dtype=object)

In [5]:
df.drop(['neo_id','name','orbiting_body'],axis=1,inplace=True)

In [6]:
df.head()

,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,relative_velocity,miss_distance,is_hazardous
0,19.14,0.394962,0.883161,71745.401048,5.814362e+07,False
1,18.50,0.530341,1.185878,109949.757148,5.580105e+07,True
2,21.45,0.136319,0.304818,24865.506798,6.720689e+07,False
3,20.63,0.198863,0.444672,78890.076805,3.039644e+07,False
4,22.70,0.076658,0.171412,56036.519484,6.311863e+07,False


In [7]:
df.duplicated().sum()

5

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 338194 entries, 0 to 338198
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   absolute_magnitude      338166 non-null  float64
 1   estimated_diameter_min  338166 non-null  float64
 2   estimated_diameter_max  338166 non-null  float64
 3   relative_velocity       338194 non-null  float64
 4   miss_distance           338194 non-null  float64
 5   is_hazardous            338194 non-null  bool   
dtypes: bool(1), float64(5)
memory usage: 15.8 MB


In [10]:
df.isnull().sum()

absolute_magnitude        28
estimated_diameter_min    28
estimated_diameter_max    28
relative_velocity          0
miss_distance              0
is_hazardous               0
dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['is_hazardous']=le.fit_transform(df['is_hazardous'])

In [12]:
df.sample(3)

,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,relative_velocity,miss_distance,is_hazardous
234318,28.40,0.005553,0.012418,22692.556791,2.655785e+07,0
273539,19.36,0.356907,0.798068,86615.068821,2.381501e+07,0
192403,23.20,0.060891,0.136157,65614.789626,4.346811e+07,0


In [13]:
X=df.drop('is_hazardous',axis=1)
y=df['is_hazardous']

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [15]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [16]:
pipeline=Pipeline(steps=[('impute',SimpleImputer(strategy='median')),('scale',StandardScaler())])

In [17]:
X_train.columns

Index(['absolute_magnitude', 'estimated_diameter_min',
       'estimated_diameter_max', 'relative_velocity', 'miss_distance'],
      dtype='object')

In [18]:
preprocessor=ColumnTransformer([('pipeline',pipeline,X_train.columns)])

In [19]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [20]:
X_train,X_test

(array([[ 1.42845498, -0.45525876, -0.45525876,  0.78520404,  0.17841367],
        [-0.38843845, -0.12961661, -0.12961661,  0.18686413, -1.52498931],
        [-0.53612544, -0.05146635, -0.05146635, -1.03967228,  0.07426034],
        ...,
        [-0.72159282,  0.07135246,  0.07135246,  1.7775149 ,  1.58661808],
        [ 0.08209917, -0.29659195, -0.29659195,  0.23668238,  0.94281846],
        [ 1.36319794, -0.4524035 , -0.4524035 , -0.79633166,  1.17933901]]),
 array([[ 1.3151138 , -0.45013387, -0.45013387,  0.38713772,  0.57584463],
        [ 0.33282359, -0.35080685, -0.35080686,  0.42305246, -0.02788028],
        [-0.5258217 , -0.05743501, -0.05743501,  0.80050905, -0.54833469],
        ...,
        [ 2.03637584, -0.47266412, -0.47266412, -1.31310437, -0.49241736],
        [-0.04498034, -0.26131593, -0.26131593,  0.32319751,  0.86489621],
        [-1.17839212,  0.54273295,  0.54273295, -1.71956171,  1.39122677]]))

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
model=Sequential()
model.add(Dense(64,activation='relu',input_shape=(X_train.shape[1],)))
model.add(Dense(32,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/opt/miniconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-08-31 07:18:18.161793: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-08-31 07:18:18.161865: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-31 07:18:18.161883: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-31 07:18:18.162007: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-31 07:18:18.162084: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created Tensor

In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,065 (15.88 KB)

 Trainable params: 4,065 (15.88 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [26]:
history=model.fit(X_train,y_train,epochs=100,batch_size=512,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/100


2024-08-31 07:18:38.765157: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


423/423 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.8686 - loss: 0.3462 - val_accuracy: 0.8846 - val_loss: 0.2417
Epoch 2/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8835 - loss: 0.2437 - val_accuracy: 0.8857 - val_loss: 0.2394
Epoch 3/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8843 - loss: 0.2409 - val_accuracy: 0.8860 - val_loss: 0.2380
Epoch 4/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8846 - loss: 0.2403 - val_accuracy: 0.8854 - val_loss: 0.2377
Epoch 5/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8845 - loss: 0.2395 - val_accuracy: 0.8862 - val_loss: 0.2371
Epoch 6/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8845 - loss: 0.2399 - val_accuracy: 0.8860 - val_loss: 0.2378
Epoch 7/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8855 - loss: 0.2376 - val_accuracy: 0.8859 - val_loss: 0.2374
Epoch 8/100
423/423 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8856 - loss: 0.2393 - val_accuracy: 0.8

In [27]:
y_pred=model.predict(X_test)

2114/2114 ━━━━━━━━━━━━━━━━━━━━ 2s 909us/step


In [28]:
y_pred=(y_pred>0.5).astype(int)

In [29]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8858942326172771

In [31]:
loss,accuracy=model.evaluate(X_test,y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

2114/2114 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8885 - loss: 0.2337
Test Loss: 0.23711398243904114
Test Accuracy: 0.885894238948822
